In [1]:

import os # 출력 경고창 없애기
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" 

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs) 
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip, num_filters):
    # 일반적인 컨볼루션의 반대 방향으로 진행 up-conv
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs) # 스트라이드가 2라서 크기가 2배로 증가 32->64
    x = Concatenate()([x, skip]) # [64x64 512, 64x64 512] -> 64x64 1024
    x = conv_block(x, num_filters) # 64x64, 1024 -> 64x64, 512
    return x

def build_unet(input_shape, num_classes):
    inputs = Input(input_shape) # 512x512 3를 받아서 케라스텐서로 변경해줌
    
    s1, p1 = encoder_block(inputs, 64) # 512, 256 
    s2, p2 = encoder_block(p1, 128) # 256, 128
    s3, p3 = encoder_block(p2, 256) # 128,64
    s4, p4 = encoder_block(p3, 512) # 64, 32

    # 플래튼 시키면 위치정보를 잃어버리니까 컨블럭에 입력
    b1 = conv_block(p4, 1024) # 32x32 1024

    # 이전형태, 저장해둔 위치정보, 채널수
    d1 = decoder_block(b1, s4, 512) # 64x64 512 
    d2 = decoder_block(d1, s3, 256) # 128x128 256
    d3 = decoder_block(d2, s2, 128) # 256x256 128
    d4 = decoder_block(d3, s1, 64) # 512x512 64

    # 마지막은 클래스 수, 커널은 1, 멀티클래스니까 softmax
    outputs = Conv2D(num_classes, 1, padding="same", activation="softmax")(d4)

    model = Model(inputs, outputs) # 모델 인스턴스 생성
    return model

# 인터프리터에서 직접 실행이 되는 경
if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_unet(input_shape, 11)
    # model.summary()

In [3]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from unet import build_unet

global image_h
global image_w
global num_classes
global classes
global rgb_codes

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*.jpg")))
    train_y = sorted(glob(os.path.join(path, "train", "labels", "*.png")))

    valid_x = sorted(glob(os.path.join(path, "val", "images", "*.jpg")))
    valid_y = sorted(glob(os.path.join(path, "val", "labels", "*.png")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.jpg")))
    test_y = sorted(glob(os.path.join(path, "test", "labels", "*.png")))

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image_mask(x, y):
    """ Image """
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (image_w, image_h))
    x = x/255.0
    x = x.astype(np.float32)

    """ Mask """
    y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    y = cv2.resize(y, (image_w, image_h))
    y = y.astype(np.int32)

    return x, y

def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()
        return read_image_mask(x, y)

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
    mask = tf.one_hot(mask, num_classes)

    image.set_shape([image_h, image_w, 3])
    mask.set_shape([image_h, image_w, num_classes])

    return image, mask

def tf_dataset(X, Y, batch=8):
    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    ds = ds.shuffle(buffer_size=5000).map(preprocess)
    ds = ds.batch(batch).prefetch(2)
    return ds

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    image_h = 512
    image_w = 512
    num_classes = 11
    input_shape = (image_h, image_w, 3)
    batch_size = 8
    lr = 1e-4 ## 0.0001
    num_epochs = 100

    """ Paths """
    dataset_path = "C:\data\lapa\LaPa"
    model_path = os.path.join("files", "model.h5")
    csv_path = os.path.join("files", "data.csv")

    """ RGB Code and Classes """
    rgb_codes = [
        [0, 0, 0], [0, 153, 255], [102, 255, 153], [0, 204, 153],
        [255, 255, 102], [255, 255, 204], [255, 153, 0], [255, 102, 255],
        [102, 0, 51], [255, 204, 255], [255, 0, 102]
    ]

    classes = [
        "background", "skin", "left eyebrow", "right eyebrow",
        "left eye", "right eye", "nose", "upper lip", "inner mouth",
        "lower lip", "hair"
    ]

    """ Loading the dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_x)}")
    print("")

    """ Dataset Pipeline """
    train_ds = tf_dataset(train_x, train_y, batch=batch_size)
    valid_ds = tf_dataset(valid_x, valid_y, batch=batch_size)

    """ Model """
    model = build_unet(input_shape, num_classes)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(lr)
    )

    """ Training """
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor='val_loss'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(train_ds,
        validation_data=valid_ds,
        epochs=num_epochs,
        callbacks=callbacks
    )




## ..

Train: 18168/18168 - Valid: 2000/2000 - Test: 2000/2000

Epoch 1/100


ResourceExhaustedError: Graph execution error:

Detected at node 'model_2/conv2d_41/Conv2D' defined at (most recent call last):
    File "c:\Users\user\miniconda3\envs\gpu2\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\user\miniconda3\envs\gpu2\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\user\miniconda3\envs\gpu2\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\user\miniconda3\envs\gpu2\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\user\miniconda3\envs\gpu2\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\user\miniconda3\envs\gpu2\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\user\miniconda3\envs\gpu2\lib\